In [4]:
!pip install tensorflow
!pip install tensorflow_gpu

  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0


ERROR: tensorflow-gpu 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow-gpu 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.


  Using cached https://files.pythonhosted.org/packages/91/2d/2ed263449a078cd9c8a9ba50ebd50123adf1f8cfbea1492f9084169b89d9/tensorboard-1.14.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3c/d5/21860a5b11caf0678fbc8319341b0ae21a07156911132e0e71bffed0510d/tensorflow_estimator-1.14.0-py2.py3-none-any.whl
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
  Found existing installation: tensorflow-estimator 1.13.0
    Uninstalling tensorflow-estimator-1.13.0:
      Successfully uninstalled tensorflow-estimator-1.13.0


ERROR: tensorflow 1.13.1 has requirement tensorboard<1.14.0,>=1.13.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 1.13.1 has requirement tensorflow-estimator<1.14.0rc0,>=1.13.0, but you'll have tensorflow-estimator 1.14.0 which is incompatible.


In [34]:
!python -m pip install --upgrade pip

  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import copy
from scipy.signal import lfilter, butter
import matplotlib.image as mpimg
from matplotlib import rcParams

from scipy import io
import seaborn as sns

from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import normalize
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn import svm
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import h5py
%matplotlib inline

# Local modules
import TEpython_Parallel

C:\Users\surface\ananew\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [ ]:
!pip uninstall tensorflow_estimator
!pip install tensorflow_estimator

In [5]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

AttributeError: module '_pywrap_tensorflow_internal' has no attribute 'TF_ListPhysicalDevices'

## Data Preprocess

<font color=pink>**Use data from 2014-05-01 to 2018-04-30**</font>

In [2]:
pathData = (r"./data")

In [3]:
# Read data table

discharge = pd.read_csv(pathData + '/'+ 'PH_ISCO_discharge_daily_cleaned.csv',header = 0,index_col = 'date',parse_dates = True, infer_datetime_format = True,low_memory=False)
discharge.column=["discharge"]
discharge.head(5)

C:\Users\surface\ananew\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


,value
date,
2014-05-01,4.56823
2014-05-02,4.54378
2014-05-03,4.97784
2014-05-04,6.07829
2014-05-05,7.20931


In [4]:
kp_soilmoisture = pd.read_csv(pathData + '/'+ 'Kettle_Ponds_moisture_daily_cleaned_fake.csv',header = 0,index_col = 'date',parse_dates = True, infer_datetime_format = True,low_memory=False)
jf_soilmoisture = pd.read_csv(pathData + '/'+ 'Judd_Falls_moisture_daily_cleaned_fake.csv',header = 0,index_col = 'date',parse_dates = True, infer_datetime_format = True,low_memory=False)

In [5]:
joined_df = discharge.merge(kp_soilmoisture, how='outer', left_index=True, right_index=True)
joined_df = joined_df.merge(jf_soilmoisture, how='outer', left_index=True, right_index=True)

joined_df.columns = ['discharge', 'kp_soilmoisture', 'jf_soilmoisture']
joined_df.head(5)

,discharge,kp_soilmoisture,jf_soilmoisture
date,,,
2014-05-01,4.56823,0.324472,0.356694
2014-05-02,4.54378,0.323333,0.356118
2014-05-03,4.97784,0.320021,0.372965
2014-05-04,6.07829,0.318597,0.388826
2014-05-05,7.20931,0.317035,0.392062


In [6]:
ncrs380_swe = pd.read_csv(pathData + '/'+ 'ncrs_380_SWE_daily_cleaned.csv',header = 0,index_col = 'date',parse_dates = True, infer_datetime_format = True,low_memory=False)
ncrs680_swe = pd.read_csv(pathData + '/'+ 'ncrs_680_SWE_daily_cleaned.csv',header = 0,index_col = 'date',parse_dates = True, infer_datetime_format = True,low_memory=False)

In [7]:
joined_df = joined_df.merge(ncrs380_swe, how='outer', left_index=True, right_index=True)
joined_df = joined_df.merge(ncrs680_swe, how='outer', left_index=True, right_index=True)
joined_df.columns = ['discharge', 'kp_soilmoisture', 'jf_soilmoisture', 'ncrs380_swe', 'ncrs680_swe']
joined_df.head(5)

,discharge,kp_soilmoisture,jf_soilmoisture,ncrs380_swe,ncrs680_swe
date,,,,,
2014-05-01,4.56823,0.324472,0.356694,26.493304,24.595667
2014-05-02,4.54378,0.323333,0.356118,25.347083,23.622000
2014-05-03,4.97784,0.320021,0.372965,23.018750,21.992167
2014-05-04,6.07829,0.318597,0.388826,20.415250,19.526250
2014-05-05,7.20931,0.317035,0.392062,17.959917,16.457083


In [8]:
ncrs380_rain = pd.read_csv(pathData + '/'+ 'ncrs_380_RAIN_daily_cleaned.csv',header = 0,index_col = 'date',parse_dates = True, infer_datetime_format = True,low_memory=False)
ncrs680_rain = pd.read_csv(pathData + '/'+ 'ncrs_680_RAIN_daily_cleaned.csv',header = 0,index_col = 'date',parse_dates = True, infer_datetime_format = True,low_memory=False)
ncrs680_rain.head(5)

,value_y
date,
2014-05-01,0.000000
2014-05-02,0.000000
2014-05-03,0.010583
2014-05-04,0.000000
2014-05-05,0.052917


In [9]:
joined_df = joined_df.merge(ncrs380_rain, how='outer', left_index=True, right_index=True)
joined_df = joined_df.merge(ncrs680_rain, how='outer', left_index=True, right_index=True)
joined_df.columns = ['discharge', 'kp_soilmoisture', 'jf_soilmoisture', 'ncrs380_swe', 'ncrs680_swe', 'ncrs380_rain', 'ncrs680_rain']
joined_df.head(5)

,discharge,kp_soilmoisture,jf_soilmoisture,ncrs380_swe,ncrs680_swe,ncrs380_rain,ncrs680_rain
date,,,,,,,
2014-05-01,4.56823,0.324472,0.356694,26.493304,24.595667,0.00000,0.000000
2014-05-02,4.54378,0.323333,0.356118,25.347083,23.622000,0.00000,0.000000
2014-05-03,4.97784,0.320021,0.372965,23.018750,21.992167,0.00000,0.010583
2014-05-04,6.07829,0.318597,0.388826,20.415250,19.526250,0.00000,0.000000
2014-05-05,7.20931,0.317035,0.392062,17.959917,16.457083,0.03175,0.052917


<font color=red>**`ncrs380_swe` and `ncrs680_swe` not enough data??**</font>

In [10]:
joined_df.count()

discharge          1461
kp_soilmoisture    1614
jf_soilmoisture    1614
ncrs380_swe        1456
ncrs680_swe        1439
ncrs380_rain       1461
ncrs680_rain       1461
dtype: int64

In [11]:
nrcs_380_temperature_clearned = pd.read_csv(pathData + '/'+ 'nrcs_380_temperature_clearned.csv',header = 0,index_col = 'datetime',parse_dates = True, infer_datetime_format = True,low_memory=False)
nrcs_680_temperature_clearned = pd.read_csv(pathData + '/'+ 'nrcs_680_temperature_clearned.csv',header = 0,index_col = 'datetime',parse_dates = True, infer_datetime_format = True,low_memory=False)
print(nrcs_380_temperature_clearned.columns)
print(nrcs_680_temperature_clearned.columns)

Index(['Unnamed: 1', 'Unnamed: 2', 'airtemperature_380'], dtype='object')
Index(['Unnamed: 1', 'Unnamed: 2', 'value'], dtype='object')


In [12]:
nrcs_380_temperature_clearned = nrcs_380_temperature_clearned.drop('Unnamed: 1', 1)
nrcs_380_temperature_clearned = nrcs_380_temperature_clearned.drop('Unnamed: 2', 1)
nrcs_680_temperature_clearned = nrcs_680_temperature_clearned.drop('Unnamed: 1', 1)
nrcs_680_temperature_clearned = nrcs_680_temperature_clearned.drop('Unnamed: 2', 1)

In [13]:
#nrcs_380_temperature_clearned.head()
nrcs_680_temperature_clearned.head()

,value
datetime,
2014-05-01,-2.1
2014-05-01,-3.6
2014-05-01,-4.2
2014-05-01,-5.4
2014-05-01,-6.9


In [14]:
# find mean group by day
nrcs_380_temperature_clearned = nrcs_380_temperature_clearned.resample('D').mean() # resamply by day/month
nrcs_380_temperature_clearned.head()

,airtemperature_380
datetime,
2014-05-01,-0.739130
2014-05-02,4.900000
2014-05-03,7.933333
2014-05-04,9.608333
2014-05-05,8.925000


In [15]:
nrcs_680_temperature_clearned = nrcs_680_temperature_clearned.resample('D').mean() # resamply by day/month
nrcs_680_temperature_clearned.head()

,value
datetime,
2014-05-01,-0.091667
2014-05-02,4.333333
2014-05-03,6.533333
2014-05-04,7.270833
2014-05-05,6.795833


In [16]:
joined_df = joined_df.merge(nrcs_380_temperature_clearned, how='outer', left_index=True, right_index=True)
joined_df = joined_df.merge(nrcs_680_temperature_clearned, how='outer', left_index=True, right_index=True)
joined_df.columns = ['discharge', 'kp_soilmoisture', 'jf_soilmoisture', 'ncrs380_swe', 'ncrs680_swe', 'ncrs380_rain', 'ncrs680_rain', 'nrcs380_temperature','nrcs680_temperature']
joined_df.head(5)

,discharge,kp_soilmoisture,jf_soilmoisture,ncrs380_swe,ncrs680_swe,ncrs380_rain,ncrs680_rain,nrcs380_temperature,nrcs680_temperature
2014-05-01,4.56823,0.324472,0.356694,26.493304,24.595667,0.00000,0.000000,-0.739130,-0.091667
2014-05-02,4.54378,0.323333,0.356118,25.347083,23.622000,0.00000,0.000000,4.900000,4.333333
2014-05-03,4.97784,0.320021,0.372965,23.018750,21.992167,0.00000,0.010583,7.933333,6.533333
2014-05-04,6.07829,0.318597,0.388826,20.415250,19.526250,0.00000,0.000000,9.608333,7.270833
2014-05-05,7.20931,0.317035,0.392062,17.959917,16.457083,0.03175,0.052917,8.925000,6.795833


In [17]:
Alldata = joined_df[0:1461]
Alldata = Alldata.fillna(value = 0)
Alldata.index.names = ['DateTime']
print(Alldata.shape)

(1461, 9)


In [20]:
Alldata

,discharge,kp_soilmoisture,jf_soilmoisture,ncrs380_swe,ncrs680_swe,ncrs380_rain,ncrs680_rain,nrcs380_temperature,nrcs680_temperature
DateTime,,,,,,,,,
2014-05-01,4.568230,0.324472,0.356694,26.493304,24.595667,0.000000e+00,0.000000e+00,-0.739130,-0.091667
2014-05-02,4.543780,0.323333,0.356118,25.347083,23.622000,0.000000e+00,0.000000e+00,4.900000,4.333333
2014-05-03,4.977840,0.320021,0.372965,23.018750,21.992167,0.000000e+00,1.058337e-02,7.933333,6.533333
2014-05-04,6.078290,0.318597,0.388826,20.415250,19.526250,0.000000e+00,0.000000e+00,9.608333,7.270833
2014-05-05,7.209310,0.317035,0.392062,17.959917,16.457083,3.174952e-02,5.291667e-02,8.925000,6.795833
2014-05-06,7.820670,0.316646,0.392493,15.546917,12.456583,1.058317e-02,6.350004e-02,7.395833,5.716667
2014-05-07,8.126350,0.317118,0.382181,13.631333,10.456333,2.010833e-01,1.799166e-01,4.241667,5.029167
2014-05-08,7.392720,0.315292,0.374437,12.467167,10.117667,1.283809e-01,8.466659e-02,-0.208333,1.120833
2014-05-09,6.261700,0.318486,0.381125,12.479131,9.038167,4.431213e-01,3.175032e-02,0.847826,1.825000


In [21]:
Alldata.to_csv('Alldata_add_temp.csv')

### Separate the loaded data into labels and features.

In [25]:
features = Alldata.iloc[:, 1:30]
labels = Alldata.iloc[:, 0]

In [23]:
features.head()

,kp_soilmoisture,jf_soilmoisture,ncrs380_swe,ncrs680_swe,ncrs380_rain,ncrs680_rain,nrcs380_temperature,nrcs680_temperature
DateTime,,,,,,,,
2014-05-01,0.324472,0.356694,26.493304,24.595667,0.00000,0.000000,-0.739130,-0.091667
2014-05-02,0.323333,0.356118,25.347083,23.622000,0.00000,0.000000,4.900000,4.333333
2014-05-03,0.320021,0.372965,23.018750,21.992167,0.00000,0.010583,7.933333,6.533333
2014-05-04,0.318597,0.388826,20.415250,19.526250,0.00000,0.000000,9.608333,7.270833
2014-05-05,0.317035,0.392062,17.959917,16.457083,0.03175,0.052917,8.925000,6.795833


In [27]:
labels.head()

DateTime
2014-05-01    4.56823
2014-05-02    4.54378
2014-05-03    4.97784
2014-05-04    6.07829
2014-05-05    7.20931
Name: discharge, dtype: float64

In [41]:
X_train,X_test,y_train,y_test = train_test_split(features, labels, test_size=0.2, shuffle=False, random_state=42)

NameError: name 'train_test_split' is not defined

## Not used PRISM attribute

In [10]:
PRISM_3_4 = pd.read_csv(pathData + '/PRISM_EastRiver_5YR_3_4.csv',
                    header = 0,index_col = 'TIME',parse_dates = True, infer_datetime_format = True)
PRISM_3_5 = pd.read_csv(pathData + '/PRISM_EastRiver_5YR_3_5.csv',
                    header = 0,index_col = 'TIME',parse_dates = True, infer_datetime_format = True)
PRISM_4_4 = pd.read_csv(pathData + '/PRISM_EastRiver_5YR_4_4.csv',
                    header = 0,index_col = 'TIME',parse_dates = True, infer_datetime_format = True)
PRISM_4_5 = pd.read_csv(pathData + '/PRISM_EastRiver_5YR_4_5.csv',
                    header = 0,index_col = 'TIME',parse_dates = True, infer_datetime_format = True)

In [11]:
PRISM_3_4.columns = PRISM_3_4.columns.str.replace('Unnamed: 0','date_int')
PRISM_3_5.columns = PRISM_3_5.columns.str.replace('Unnamed: 0','date_int')
PRISM_4_4.columns = PRISM_4_4.columns.str.replace('Unnamed: 0','date_int')
PRISM_4_5.columns = PRISM_4_5.columns.str.replace('Unnamed: 0','date_int')
PRISM_3_4 = PRISM_3_4[119:1580]
PRISM_3_5 = PRISM_3_5[119:1580]
PRISM_4_4 = PRISM_4_4[119:1580]
PRISM_4_5 = PRISM_4_5[119:1580]

In [12]:
PRISM_precp_34 = PRISM_3_4['PRECT']
PRISM_precp_35 = PRISM_3_5['PRECT']
PRISM_precp_44 = PRISM_4_4['PRECT']
PRISM_precp_45 = PRISM_4_5['PRECT']

In [13]:
PRISM = pd.read_csv(pathData + '/'+ 'PRISM.csv',header = 0,index_col = 'date',parse_dates = True, infer_datetime_format = True,low_memory=False)
PRISM.head(5)

,PRISM34,PRISM35,PRISM44,PRISM45
date,,,,
2014-05-01,5.09,8.09,9.75,10.52
2014-05-02,2.44,4.70,4.97,6.22
2014-05-03,0.40,1.74,4.50,5.76
2014-05-04,0.04,0.94,2.03,3.56
2014-05-05,0.00,0.00,0.00,0.00


In [14]:
joined_df = joined_df.merge(PRISM, how='outer', left_index=True, right_index=True)
#joined_df.columns = ['discharge', 'kp_soilmoisture', 'jf_soilmoisture', 'ncrs380_swe', 'ncrs680_swe', 'ncrs380_rain', 'ncrs680_rain', 'PRISM']
joined_df.head(5)

,discharge,kp_soilmoisture,jf_soilmoisture,ncrs380_swe,ncrs680_swe,ncrs380_rain,ncrs680_rain,PRISM34,PRISM35,PRISM44,PRISM45
date,,,,,,,,,,,
2014-05-01,4.56823,0.324472,0.356694,26.493304,24.595667,0.00000,0.000000,5.09,8.09,9.75,10.52
2014-05-02,4.54378,0.323333,0.356118,25.347083,23.622000,0.00000,0.000000,2.44,4.70,4.97,6.22
2014-05-03,4.97784,0.320021,0.372965,23.018750,21.992167,0.00000,0.010583,0.40,1.74,4.50,5.76
2014-05-04,6.07829,0.318597,0.388826,20.415250,19.526250,0.00000,0.000000,0.04,0.94,2.03,3.56
2014-05-05,7.20931,0.317035,0.392062,17.959917,16.457083,0.03175,0.052917,0.00,0.00,0.00,0.00


In [15]:
Alldata = joined_df[0:1461]
Alldata = Alldata.fillna(value = 0)
Alldata.index.names = ['DateTime']
print(Alldata.shape)

(1461, 11)


In [16]:
Alldata.head(5)

,discharge,kp_soilmoisture,jf_soilmoisture,ncrs380_swe,ncrs680_swe,ncrs380_rain,ncrs680_rain,PRISM34,PRISM35,PRISM44,PRISM45
DateTime,,,,,,,,,,,
2014-05-01,4.56823,0.324472,0.356694,26.493304,24.595667,0.00000,0.000000,5.09,8.09,9.75,10.52
2014-05-02,4.54378,0.323333,0.356118,25.347083,23.622000,0.00000,0.000000,2.44,4.70,4.97,6.22
2014-05-03,4.97784,0.320021,0.372965,23.018750,21.992167,0.00000,0.010583,0.40,1.74,4.50,5.76
2014-05-04,6.07829,0.318597,0.388826,20.415250,19.526250,0.00000,0.000000,0.04,0.94,2.03,3.56
2014-05-05,7.20931,0.317035,0.392062,17.959917,16.457083,0.03175,0.052917,0.00,0.00,0.00,0.00


In [18]:
Alldata.to_csv('Alldata.csv')